In [2]:
import scipy
import scipy.stats as scstats
import scipy.special as sps
import numpy as np

import matplotlib.pyplot as plt
import daft

import sys
sys.path.append("..")
sys.path.append("../scripts/") # access to scripts


import os
if not os.path.exists('./output'):
    os.makedirs('./output')
    

from neuroprob import utils

import model_utils

import pickle



plt.style.use(['paper.mplstyle'])

### Plotting data loading

In [ ]:
datarun = pickle.load(open('./saves/P_HDC_rg40.p', 'rb'))

avg_models, var_models, ff_models, \
    Pearson_ff, ratio, \
    PLL_rg_ll, PLL_rg_cov, \
    Qq_ll, Zz_ll, R_ll, Rp_ll, q_DS_ll, T_DS_ll, T_KS_ll, \
    sign_KS, sign_DS, \
    mhd_mean, mhd_ff, hd_mean_tf, hd_ff_tf, \
    mw_mean, mw_ff, w_mean_tf, w_ff_tf, \
    ms_mean, ms_ff, s_mean_tf, s_ff_tf, \
    mt_mean, mt_ff, t_mean_tf, t_ff_tf, \
    mpos_mean, mpos_ff, pos_mean_tf, pos_ff_tf, \
    covariates_hd, lower_hd, mean_hd, upper_hd, \
    fflower_hd, ffmean_hd, ffupper_hd, \
    covariates_s, lower_s, mean_s, upper_s, \
    fflower_s, ffmean_s, ffupper_s, \
    covariates_t, lower_t, mean_t, upper_t, \
    fflower_t, ffmean_t, ffupper_t, \
    covariates_w, lower_w, mean_w, upper_w, \
    fflower_w, ffmean_w, ffupper_w, \
    grid_size_pos, grid_shape_pos, field_pos, ff_pos, \
    grid_size_hdw, grid_shape_hdw, field_hdw, \
    grid_size_hdt, grid_shape_hdt, field_hdt, \
    pref_hdw, ATI, res_var, \
    pref_hdt, drift, res_var_drift, \
    tun_width, amp_t, ampm_t, sim_mat, \
    pick_neuron, max_count, tbin, rcov, region_edge = datarun


datarun = pickle.load(open('./saves/P_HDC_nc40.p', 'rb'))

avg_models_z, var_models_z, ff_models_z, \
    Pearson_ffz, ratioz, \
    X_c, X_s, cv_pll, elbo, z_tau, pref_hd,\
    grid_size_zz, grid_shape_zz, field_zz, ff_zz, \
    mz1_mean, mz1_ff, z1_mean_tf, z1_ff_tf, \
    mz2_mean, mz2_ff, z2_mean_tf, z2_ff_tf, \
    q_DS_, T_DS_, T_KS_, Qq, Zz, R, Rp, fisher_z, fisher_q, \
    T_KS_fishq, p_KS_fishq, T_KS_ks, p_KS_ks, \
    R_mat_spt, R_mat_sptp, \
    timescales, acg_rc, acg_z, t_lengths = datarun


datarun = pickle.load(open('./saves/P_HDC_lat.p', 'rb'))

lat_t, lat_t_std, delay_RMS, RMS_cv, LVM_cv_ll, drifts_lv, rcov_lvm = datarun

region_edge = region_edge[0] # boundary separating PoS (lower) and ANT (higher or equal)

### Plot figure 3

In [ ]:
fig = plt.figure(figsize=(8, 4))
fig.text(-0.08, 1.02, 'A', transform=fig.transFigure, size=15, fontweight='bold')
fig.text(0.415, 1.02, 'B', transform=fig.transFigure, size=15, fontweight='bold')
fig.text(-0.08, 0.6, 'C', transform=fig.transFigure, size=15, fontweight='bold')
fig.text(-0.08, -0.15, 'D', transform=fig.transFigure, size=15, fontweight='bold')
fig.text(0.23, -0.15, 'E', transform=fig.transFigure, size=15, fontweight='bold')
fig.text(0.565, -0.15, 'F', transform=fig.transFigure, size=15, fontweight='bold')
fig.text(0.87, -0.15, 'G', transform=fig.transFigure, size=15, fontweight='bold')


white = '#ffffff'
black = '#000000'
red = '#ff0000'
blue = '#0000ff'
weight_map = utils.plot.make_cmap([blue, white, red], 'weight_map')



poscol = 'forestgreen'
antcol = 'orange'
BINS = 20
S = 2 # marker size


H = grid_shape_pos[1][1]-grid_shape_pos[1][0]
W = grid_shape_pos[0][1]-grid_shape_pos[0][0]


show_neuron = [11, 26] # PoS and ANT respectively

### regression ###
regression_scores(fig)
binning_stats(fig)
regression_stats(fig)
tunings(fig)

### noise correlations ###
noise_correlation_scores(fig)
noise_correlations_mats(fig)
noise_correlations_single_neuron_variability(fig)
latent_trajs(fig)
latent_timescales(fig)


plt.savefig('output/plot_hdc.pdf')
plt.show()

Some extra analysis below here

In [ ]:
# one sample t-test, cv_pll has shape (model, cv_trial, neuron_group)
c = cv_pll.mean(-1)*(len(pick_neuron)/5) # mean over neuron cv groups, rescale to be comparable to A
rel_c = (c-c[0:1, :])
c_ = rel_c.mean(-1)
yerr = rel_c.std(1, ddof=1)/np.sqrt(rel_c.shape[1])

ttest = []
for rc, rce in zip(c_[1:], yerr[1:]):
    ttest.append(scipy.stats.ttest_1samp(rc, 0))

In [ ]:
# check KS statistic of Fisher Z scores for different latent dimensions, compare to p=0.005 line
plt.plot(np.log(p_KS_fishq[0]), marker='.')
plt.plot(np.arange(5), np.log(np.ones(5)*0.005))

In [ ]:
# one-sample t t-test for regression baseline of Universal versus Poisson
order = [1, 2, 0]
scores = PLL_rg_ll
rel_score = (scores[order, :]-scores[order, :][0:1, :])
score_err = rel_score.std(-1, ddof=1)/np.sqrt(rel_score.shape[-1])

scipy.stats.ttest_1samp(rel_score[2, :], 0)

In [ ]:
# plot the T_DS statistic for all latent dimensions

fig = plt.figure(figsize=(8, 4))

L = 0.04
Ncases = T_DS_.shape[1]
kcv_ind = 0 # choose validation set segment
widths = np.ones(Ncases)
heights = [1]
spec = fig.add_gridspec(ncols=len(widths), nrows=len(heights), width_ratios=widths,
                        height_ratios=heights, wspace=0.4, 
                        left=.0+X, right=0.2+X, bottom=-0.5+Y, top=-.35+Y)

# T_DS_.shape is (cv, z_dim, neurons, 1)
for en, r in enumerate(T_DS_[kcv_ind][..., 0]):
    ax = fig.add_subplot(spec[0, en])
    
    ax.hist(r, bins=np.linspace(-L, L, 20), density=True, orientation='horizontal', 
            color='gray')
    ax.set_xticks([])
    if en == 0:
        ax.set_ylabel(r'$T_{DS}$', labelpad=4, fontsize=10)
    else:
        ax.set_yticks([])
    
    samples = len(Zz[0,0,0])
    std = np.sqrt(2/(samples-1))
    xx = np.linspace(-L, L, 100)
    yy = scstats.norm.pdf(xx/std)/std
    ax.plot(yy, xx, 'r--')
    ax.set_ylim(-L, L)
    ax.set_xlim(0, yy.max()*1.2)
    ax.set_xlabel('{}'.format(en))
    

plt.show()

In [ ]:
# Show the empirical distribution of the KS statistic, T_KS_.shape is (cv, z_dim, neurons, 1)
T = T_KS_[0, 0, :, 0]
x = np.linspace(0, 0.01, 100)
f = utils.stats.KS_sampling_dist(x, len(Qq[0, 0, 0]))
plt.plot(x, f)
plt.hist(T, density=True)
plt.show()
    

In [ ]:
# show some tuning curves to latent dimensions

fig = plt.figure(figsize=(8, 6))



X = 0.0
Y = 0.0
# tuning
widths = np.ones(len(show_neuron))
heights = [1, 1]
spec = fig.add_gridspec(ncols=len(widths), nrows=len(heights), width_ratios=widths,
                        height_ratios=heights, hspace=0.5, wspace=0.2, 
                        left=.51+X, right=0.73+X, bottom=-0.37+Y, top=Y-0.15)

for k, ne in enumerate(show_neuron):
    ax = fig.add_subplot(spec[0, k])
    
    rate = field_zz[ne]/tbin
    ax.set_title('{:.1f} Hz'.format(rate.max()), fontsize=10, pad=-5)
    im = utils.plot.visualize_field((fig, ax), rate.T, grid_shape_pos, cbar=False, aspect='auto')
    utils.plot.decorate_ax(ax, spines=[False, False, False, False])
    
bp = 3/4*(grid_shape_zz[0][1]-grid_shape_zz[0][0]) + grid_shape_zz[0][0]
rp = 1/4*(grid_shape_zz[0][1]-grid_shape_zz[0][0]) + grid_shape_zz[0][0]
py = 1.2*grid_shape_zz[1][1]
for k, ne in enumerate(show_neuron):
    ax = fig.add_subplot(spec[1, k])
    
    FF = ff_zz[ne]
    rate = np.log(FF)
    g = max(-rate.min(), rate.max())
    ax.text(rp, py, '{:.1f}'.format(np.exp(g)), ha='center', fontsize=10, color='red')
    ax.text(bp, py, '{:.1f}'.format(np.exp(-g)), ha='center', fontsize=10, color='blue')
    im = utils.plot.visualize_field((fig, ax), rate.T, grid_shape_zz, cbar=False, aspect='auto', 
                                    vmin=-g, vmax=g, cmap=weight_map)
    utils.plot.decorate_ax(ax, spines=[False, False, False, False])
    
    if k == 0:
        ax.set_xticks(grid_shape_zz[0])
        ax.set_yticks(grid_shape_zz[1])
    
fig.text(0.45+X, Y-0.25, r'$z_2$', rotation=90, fontsize=10, va='center')
fig.text(0.62+X, Y-0.42, r'$z_1$', fontsize=10, ha='center')


In [ ]:
# plot the noise correlation magnitudes against preferred head direction difference

order = np.argsort(pref_hdw[:, 10])#list(np.argsort(pref_hdw[:region_edge, 10])) + list(np.argsort(pref_hdw[region_edge:, 10])+region_edge)

hd = np.abs((pref_hdw[order, 10][:, None] - pref_hdw[order, 10][None, :]) % (np.pi))
rd = R_mat_spt[0, 0][order, :][:, order]

plt.scatter(np.tril(hd, k=-1).flatten(), np.tril(rd, k=-1).flatten())

In [ ]:
# CCG of input dimensions
delays = 5000
Tsteps = rcov[0].shape[0]
L = Tsteps-delays+1


rc_tot = list(rcov[:-1]) + list(X_c.T)
ccg = []

L = Tsteps-2*delays
for en, rc in enumerate(rc_tot):
    for rc_ in rc_tot[en+1:]:
        cg = np.empty(2*delays+1)
        for d in range(-delays, delays+1):
            A = rc[d+delays:d+delays+L]
            B = rc_[delays:delays+L]

            if en == 0:
                cg[d+delays] = utils.stats.corr_lin_circ(B, A)
            else:
                cg[d+delays] = utils.stats.corr_lin_lin(A, B)
                
        ccg.append(cg)

In [ ]:
# cross correlations between behavioural and latent trajectories
fig = plt.figure(figsize=(8, 8))



RC_n = len(rc_tot)
Tps = [1000, 100, 100, 3000, 3000, 100, 100]

widths = np.ones(RC_n)
heights = np.ones(RC_n)
spec = fig.add_gridspec(ncols=len(widths), nrows=len(heights), width_ratios=widths,
                        height_ratios=heights, hspace=0.7, wspace=0.7, 
                        left=.0, right=1., bottom=0., top=1.)

for n in range(RC_n):
    ax = fig.add_subplot(spec[n, n])
    Tp = Tps[n]
    t = np.arange(Tp)*tbin
    if n >= len(rcov[:-1]):
        ax.plot(t, acg_z[n-len(rcov[:-1])][:Tp])
    else:
        ax.plot(t, acg_rc[n][:Tp])
        
    if n == 0:
        ax.set_ylabel('HD')
        ax.set_xlabel('time (s)')
    elif n == RC_n-1:
        ax.set_xlabel(r'$z_2$')
        
        
labels = ['HD', 'AHV', 'speed', r'$x$', r'$y$', r'$z_1$', r'$z_2$']
Tp_ = 5000
n__ = 0
for n in range(RC_n):
    for n_ in range(n+1, RC_n):
        ax = fig.add_subplot(spec[n_, n])
        t = np.arange(-Tp_, Tp_+1)*tbin
        ax.plot(t, ccg[n__][:])
        n__ += 1
        
        if n == 0:
            ax.set_ylabel(labels[n_])
        if n_ == RC_n-1:
            ax.set_xlabel(labels[n])
        
        
plt.show()